In [37]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_lgb
importlib.reload(preprocesamiento)
importlib.reload(model_lgb)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

# Experimento 7: 
- LGBM
- Usando funcion entrenamiento: model_lgb.semillerio_en_prediccion(train, test, version="v1")
- optuna = sqlite:///optuna_studies_v14.db
- Kaggle =  


##### Levantamos el dataset con target ya calculado

In [3]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
df.shape

(31362, 19)

In [4]:
columnas_baseline = df.columns.tolist()
columnas_baseline

['product_id',
 'periodo',
 'nacimiento_producto',
 'muerte_producto',
 'mes_n',
 'total_meses',
 'producto_nuevo',
 'ciclo_de_vida_inicial',
 'cat1',
 'cat2',
 'cat3',
 'brand',
 'sku_size',
 'stock_final',
 'tn',
 'plan_precios_cuidados',
 'cust_request_qty',
 'cust_request_tn',
 'target']

##### Preprocesamiento a la minima expresión :)

In [50]:
# ##### aplicamos OHE
# df = preprocesamiento.aplicarOHE(df)
# df.shape

### Feature Engineering

##### Neural Prophet

In [5]:
neural_prophet_fe = pd.read_csv("./datasets/features_neuralprophet_completo.csv", sep=',', encoding='utf-8')
neural_prophet_fe['ds'] = pd.to_datetime(neural_prophet_fe['ds'], errors='coerce')
# Versión alternativa más robusta:
neural_prophet_fe['periodo'] = neural_prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
neural_prophet_fe = neural_prophet_fe[['periodo', 'product_id', 'trend', "season_yearly", "season_monthly"]]
df = df.merge(neural_prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 22)

##### Prophet

In [6]:
prophet_fe = pd.read_csv("./datasets/prophet_features_tn_zscore.csv", sep=',', encoding='utf-8')
prophet_fe['ds'] = pd.to_datetime(prophet_fe['ds'], errors='coerce')
prophet_fe['periodo'] = prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
prophet_fe = prophet_fe[['periodo', 'product_id', 'trend_add', "yearly_add", "additive_terms", 'trend_mult', 'yearly_mult', 'multiplicative_terms']]
df = df.merge(prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 28)

##### FE Moviles

In [7]:
df = feature_engineering.get_lags(df, "tn", 201912)
df = feature_engineering.get_delta_lags(df, "tn", 24)
df = feature_engineering.get_rolling_means(df, "tn", 201912)
df = feature_engineering.get_rolling_stds(df, "tn", 201912)
df = feature_engineering.get_rolling_mins(df, "tn", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn", 201912)
df = feature_engineering.get_rolling_medians(df, "tn", 201912)
df = feature_engineering.get_rolling_skewness(df, "tn", 201912)
df = feature_engineering.get_autocorrelaciones(df, "tn", 201912)
df.shape

(31362, 590)

In [8]:
df = feature_engineering.get_lags(df, "cust_request_qty", 201912)
df = feature_engineering.get_delta_lags(df, "cust_request_qty", 24)
df = feature_engineering.get_rolling_means(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_stds(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_mins(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_maxs(df, "cust_request_qty", 201912)

In [ ]:
# df = feature_engineering.get_lags(df, "stock_final", 201912)
# df = feature_engineering.get_delta_lags(df, "stock_final", 24)
# df = feature_engineering.get_rolling_means(df, "stock_final", 201912)
# df = feature_engineering.get_rolling_stds(df, "stock_final", 201912)
# df = feature_engineering.get_rolling_mins(df, "stock_final", 201912)
# df = feature_engineering.get_rolling_maxs(df, "stock_final", 201912)
# df.shape

(31362, 1500)

##### FE Moviles sobre otras variables

In [56]:
# #  stock final
# df = feature_engineering.get_lagsEspecificos(df, col='stock_final_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='stock_final_zscore')

#  cust_request_qty
# df = feature_engineering.get_lagsEspecificos(df, col='cust_request_qty')
# df = feature_engineering.get_delta_lags_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_means_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_stds_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_mins_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_medians_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='cust_request_qty')

##### FE Calendario

In [9]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)
df = feature_engineering.get_anomaliasPoliticas(df)
# df = feature_engineering.descomposicion_serie_temporal(df, col='tn')
df.shape

(31362, 1070)

##### FE sobre FE

In [10]:
df = feature_engineering.chatGPT_features_serie(df, "tn")
df = feature_engineering.mes_con_feriado(df)
df.shape

(31362, 1099)

##### Variables Exogenas

In [11]:
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_IPC(df)
df['ipc'] = df['ipc'].str.replace(',', '.').astype(float)
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)
# df.drop(columns=['ds'], inplace=True)
df.fillna(0, inplace=True) ##### EXPERIMENTAR
df = feature_engineering.correlacion_exogenas(df)
df = feature_engineering.get_mes_receso_escolar(df)
df.shape

(31362, 1102)

##### Nuevas FE

In [12]:
df = feature_engineering.create_ratio_features(df)
df = feature_engineering.enhance_lifecycle_features(df)
df = feature_engineering.create_category_features(df)
df = feature_engineering.create_regime_features(df)
df = feature_engineering.create_nonlinear_trends(df)
df = feature_engineering.create_temporal_interactions(df)
df = feature_engineering.create_asymmetric_window_features(df)
df = feature_engineering.recomendaciones_deepseek(df)
df = feature_engineering.get_nuevas_features(df)
df.shape

(31362, 1144)

##### Elimino aquellas que no sirven

In [13]:
importantes = pd.read_csv("./feature_importance/exp04_3.csv", sep=',', encoding='utf-8')
no_importantes = importantes[importantes['importance'] <= 100]
no_importantes = no_importantes[~no_importantes['feature'].isin(columnas_baseline)]
no_importantes

,feature,importance
67,tn_vs_prev_year,100
68,tn_delta_lag5_lag8,99
69,tn_rolling_min_4,96
70,tn_lag_5,95
71,tn_rolling_std_6,95
...,...,...
677,dia_del_year,0
678,cat2_TE,0
680,cat2_PIEL1,0
681,cat2_OTROS,0


In [14]:
cols_a_eliminar = no_importantes.feature.unique()
print(f"Antes de eliminar: {df.shape[1]} columnas")
df = df.drop(columns=cols_a_eliminar, errors='ignore')
print(f"Después de eliminar: {df.shape[1]} columnas")

Antes de eliminar: 1144 columnas
Después de eliminar: 684 columnas


Eliminar object/categorical columnas

In [15]:
df = df.select_dtypes(exclude=['datetime', 'datetime64', 'object'])

Train Test Split

In [16]:
train = df[df['periodo'] <= 201912]
test = df[df['periodo'] == 201912]

Entrenamiento

In [35]:
model_lgb.optimizar_con_optuna_con_semillerio_db(train, version="v14")

[I 2025-06-29 22:19:14,389] Using an existing study with name 'lightgbm_optimization_v14' instead of creating a new one.



Para visualizar los resultados en tiempo real:
1. Abre otra terminal y ejecuta:
   optuna-dashboard sqlite:///optuna_studies_v14.db
2. Abre en tu navegador: http://127.0.0.1:8080/


[I 2025-06-29 22:24:35,898] Trial 16 finished with value: 12.889061597019383 and parameters: {'num_leaves': 81, 'learning_rate': 0.016642751070396707, 'feature_fraction': 0.6030189277265172, 'bagging_fraction': 0.7053885626844458, 'bagging_freq': 9, 'lambda_l1': 0.016301353379407614, 'lambda_l2': 4.3444691085504505, 'min_child_samples': 11, 'max_depth': 10, 'max_bin': 104, 'min_data_in_leaf': 58, 'extra_trees': True, 'early_stopping_rounds': 47, 'path_smooth': 0.07644843397269116, 'min_gain_to_split': 0.4555376912834475}. Best is trial 6 with value: 0.001158291164702125.


Mejor trial hasta ahora: RMSE=0.001158, Parámetros={'num_leaves': 59, 'learning_rate': 0.042808491617570936, 'feature_fraction': 0.610167650697638, 'bagging_fraction': 0.7323674280979913, 'bagging_freq': 1, 'lambda_l1': 0.005341874754868531, 'lambda_l2': 6.748446817464346e-06, 'min_child_samples': 30, 'max_depth': 10, 'max_bin': 199, 'min_data_in_leaf': 53, 'extra_trees': True, 'early_stopping_rounds': 13, 'path_smooth': 0.289751452913768, 'min_gain_to_split': 0.08061064362700221}


[I 2025-06-29 22:27:40,613] Trial 17 finished with value: 1.3439178191178622 and parameters: {'num_leaves': 68, 'learning_rate': 0.14027758719310485, 'feature_fraction': 0.8690135987041901, 'bagging_fraction': 0.8535675375405839, 'bagging_freq': 1, 'lambda_l1': 0.0006140317027634772, 'lambda_l2': 1.6535721007135704e-08, 'min_child_samples': 39, 'max_depth': 8, 'max_bin': 488, 'min_data_in_leaf': 54, 'extra_trees': True, 'early_stopping_rounds': 24, 'path_smooth': 0.07846238496678071, 'min_gain_to_split': 0.0016499476527944285}. Best is trial 6 with value: 0.001158291164702125.


...

[I 2025-06-30 01:15:50,602] Trial 68 finished with value: 98.462107295068 and parameters: {'num_leaves': 47, 'learning_rate': 0.04720424818788646, 'feature_fraction': 0.7025960550695486, 'bagging_fraction': 0.8682425119392283, 'bagging_freq': 8, 'lambda_l1': 5.0905136911883744e-08, 'lambda_l2': 1.2362711673959513e-06, 'min_child_samples': 50, 'max_depth': 3, 'max_bin': 268, 'min_data_in_leaf': 32, 'extra_trees': False, 'early_stopping_rounds': 13, 'path_smooth': 0.8205641855175438, 'min_gain_to_split': 0.04376490870753164}. Best is trial 6 with value: 0.001158291164702125.


Mejor trial hasta ahora: RMSE=0.001158, Parámetros={'num_leaves': 59, 'learning_rate': 0.042808491617570936, 'feature_fraction': 0.610167650697638, 'bagging_fraction': 0.7323674280979913, 'bagging_freq': 1, 'lambda_l1': 0.005341874754868531, 'lambda_l2': 6.748446817464346e-06, 'min_child_samples': 30, 'max_depth': 10, 'max_bin': 199, 'min_data_in_leaf': 53, 'extra_trees': True, 'early_stopping_rounds': 13, 'path_smooth': 0.289751452913768, 'min_gain_to_split': 0.08061064362700221}


[I 2025-06-30 01:19:26,832] Trial 69 finished with value: 14.568964518218905 and parameters: {'num_leaves': 41, 'learning_rate': 0.09850655661617769, 'feature_fraction': 0.9245826779918879, 'bagging_fraction': 0.7634521063793515, 'bagging_freq': 8, 'lambda_l1': 5.131445266883285e-06, 'lambda_l2': 7.952049505896054e-05, 'min_child_samples': 24, 'max_depth': 7, 'max_bin': 326, 'min_data_in_leaf': 42, 'extra_trees': False, 'early_stopping_rounds': 10, 'path_smooth': 0.6251530489206609, 'min_gain_to_split': 0.017272593884831544}. Best is trial 6 with value: 0.001158291164702125.


Mejor trial hasta ahora: RMSE=0.001158, Parámetros={'num_leaves': 59, 'learning_rate': 0.042808491617570936, 'feature_fraction': 0.610167650697638, 'bagging_fraction': 0.7323674280979913, 'bagging_freq': 1, 'lambda_l1': 0.005341874754868531, 'lambda_l2': 6.748446817464346e-06, 'min_child_samples': 30, 'max_depth': 10, 'max_bin': 199, 'min_data_in_leaf': 53, 'extra_trees': True, 'early_stopping_rounds': 13, 'path_smooth': 0.289751452913768, 'min_gain_to_split': 0.08061064362700221}
Estudio guardado en: sqlite:///optuna_studies_v14.db

Mejores hiperparámetros encontrados:
num_leaves: 59
learning_rate: 0.042808491617570936
feature_fraction: 0.610167650697638
bagging_fraction: 0.7323674280979913
bagging_freq: 1
lambda_l1: 0.005341874754868531
lambda_l2: 6.748446817464346e-06
min_child_samples: 30
max_depth: 10
max_bin: 199
min_data_in_leaf: 53
extra_trees: True
early_stopping_rounds: 13
path_smooth: 0.289751452913768
min_gain_to_split: 0.08061064362700221


(<optuna.study.study.Study at 0x15646bf5dd0>,
 {'num_leaves': 59,
  'learning_rate': 0.042808491617570936,
  'feature_fraction': 0.610167650697638,
  'bagging_fraction': 0.7323674280979913,
  'bagging_freq': 1,
  'lambda_l1': 0.005341874754868531,
  'lambda_l2': 6.748446817464346e-06,
  'min_child_samples': 30,
  'max_depth': 10,
  'max_bin': 199,
  'min_data_in_leaf': 53,
  'extra_trees': True,
  'early_stopping_rounds': 13,
  'path_smooth': 0.289751452913768,
  'min_gain_to_split': 0.08061064362700221,
  'objective': 'regression',
  'metric': 'rmse',
  'boosting_type': 'gbdt',
  'verbosity': -1})

Prediccion

In [38]:
df_future = model_lgb.semillerio_en_prediccion_con_pesos(train, test, version="v1")

In [39]:
df_future

,periodo,product_id,target,pred
30476,201912,20001,0.0,1490.523819
30477,201912,20002,0.0,1176.601177
30478,201912,20003,0.0,760.653373
30479,201912,20004,0.0,713.971133
30480,201912,20005,0.0,610.816894
...,...,...,...,...
31357,201912,21265,0.0,-8.235051
31358,201912,21266,0.0,-8.913198
31359,201912,21267,0.0,-5.841841
31360,201912,21271,0.0,-1.394455


In [ ]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
df_future = df_future[df_future['periodo'] == 201912]
df_future = df_future[df_future['product_id'].isin(productos_ok['product_id'].unique())]


,periodo,product_id,target,pred
30476,201912,20001,0.0,1490.523819
30477,201912,20002,0.0,1176.601177
30478,201912,20003,0.0,760.653373
30479,201912,20004,0.0,713.971133
30480,201912,20005,0.0,610.816894
...,...,...,...,...
31355,201912,21263,0.0,-6.386380
31357,201912,21265,0.0,-8.235051
31358,201912,21266,0.0,-8.913198
31359,201912,21267,0.0,-5.841841


In [ ]:
linear_regression = pd.read_csv("./outputs/predicciones_regresion_lineal_v2.csv", sep=',', encoding='utf-8')
df_future = df_future[['product_id', 'pred']]
df_future = df_future.merge(linear_regression, on='product_id', how='left')
df_future.loc[df_future['pred'] < 0, 'pred'] = df_future['tn']
# Definir el rango de product_ids
mask = (df_future['product_id'] >= 20001) & (df_future['product_id'] <= 20005)

# Asignar el valor de tn a pred para esos productos
df_future.loc[mask, 'pred'] = df_future.loc[mask, 'tn']
df_future



,product_id,pred,tn
0,20001,1232.764336,1232.764336
1,20002,1135.499014,1135.499014
2,20003,683.869748,683.869748
3,20004,551.448542,551.448542
4,20005,546.346434,546.346434
...,...,...,...
775,21263,0.249044,0.249044
776,21265,0.072234,0.072234
777,21266,0.076151,0.076151
778,21267,0.075869,0.075869


In [45]:
df_future.drop(columns=['tn'], inplace=True)
df_future.rename(columns={'pred': 'tn'}, inplace=True)
df_future.to_csv("./outputs/predicciones_exp_07_lgb_v1.csv", index=False, sep=',')

Feature importance

In [78]:
model_lgb.feature_importance(df, models, "05")

In [79]:
importances = model_lgb.feature_importance_promedio(models, df.drop(columns=['target']))
importances.sort_values(by='importance', ascending=False, inplace=True) 
importances

,feature,importance
16,yearly_add,0.025628
0,product_id,0.017944
1084,tn_seasonal_naive,0.014613
1119,naive_forecast_error,0.014388
1080,tn_ytd_sum,0.012998
...,...,...
1053,stock_final_rolling_max_18,0.000000
1052,stock_final_rolling_max_17,0.000000
1051,stock_final_rolling_max_16,0.000000
1050,stock_final_rolling_max_15,0.000000
